In [15]:

import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer



In [16]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure nltk data is downloaded
nltk.data.path.append(r"C:\Users\ASHISHI RANJAN\AppData\Roaming\nltk_data")
nltk.download('punkt', force=True)
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    tokens = word_tokenize(text)  # Tokenize
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token.lower() not in stop_words]  # Remove stopwords
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
    return ' '.join(tokens)

[nltk_data] Downloading package punkt to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
print(nltk.data.find('tokenizers/punkt'))


C:\Users\ASHISHI RANJAN\AppData\Roaming\nltk_data\tokenizers\punkt


In [18]:
# def preprocess_text(text):
#     # Tokenize text
#     tokens = word_tokenize(text)

#     # Remove stop words
#     stop_words = set(stopwords.words('english'))
#     tokens = [token for token in tokens if token.lower() not in stop_words]

#     # Lemmatize tokens
#     lemmatizer = WordNetLemmatizer()
#     tokens = [lemmatizer.lemmatize(token) for token in tokens]

#     return ' '.join(tokens)

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

def text_representation(text):
    vectorizer = TfidfVectorizer()
    vectorizer.fit([preprocess_text(text)])  # Fit TF-IDF
    vectors = vectorizer.transform([preprocess_text(text)])  # Transform text to vector
    return vectors

In [20]:
# from sumy.parsers.plaintext import PlaintextParser
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lsa import LsaSummarizer

# def summarize_text(text):
#     parser = PlaintextParser.from_string(text, Tokenizer("english"))
#     summarizer = LsaSummarizer()
#     summary = summarizer(parser.document, 3)  # Number of sentences
#     return " ".join(str(sentence) for sentence in summary)


In [21]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_text(text, min_words=50, max_words=200):
    # Generate summary with BART while ensuring word constraints
    raw_summary = summarizer(text, max_length=max_words, min_length=min_words, do_sample=False)[0]['summary_text']

    # Trim summary to exact word range
    words = raw_summary.split()
    num_words = min(len(words), max(1, (min_words + max_words) // 2))  # Balanced output
    final_summary = " ".join(words[:num_words])

    return final_summary


All PyTorch model weights were used when initializing TFBartForConditionalGeneration.

All the weights of TFBartForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBartForConditionalGeneration for predictions without further training.
Device set to use 0


In [22]:
def post_process_summary(summary):
    # Remove extra spaces
    summary = ' '.join(summary.split())

    return summary

In [23]:
text = """NLTK (Natural Language Toolkit) is a popular Python library for natural language processing (NLP). It provides us various text processing libraries with a lot of test datasets. A variety of tasks can be performed using NLTK such as tokenizing, parse tree visualization, etc… In this article, we will go through how we can set up NLTK in our system and use them for performing various NLP tasks during the text processing step. Tokenization:
The breaking down of text into smaller units is called tokens. tokens are a small part of that text. If we have a sentence, the idea is to separate each word and build a vocabulary such that we can represent all words uniquely in a list. Numbers, words, etc.. all fall under tokens.

Lower case conversion:
We want our model to not get confused by seeing the same word with different cases like one starting with capital and one without and interpret both differently. So we convert all words into the lower case to avoid redundancy in the token list. Stop Words removal:
When we use the features from a text to model, we will encounter a lot of noise. These are the stop words like the, he, her, etc… which don’t help us and, just be removed before processing for cleaner processing inside the model. With NLTK we can see all the stop words available in the English language.

from nltk.corpus import stopwords"""
summary = summarize_text(text)
print("Final Summary:", post_process_summary(summary))

Final Summary: NLTK (Natural Language Toolkit) is a popular Python library for natural language processing (NLP) It provides us various text processing libraries with a lot of test datasets. A variety of tasks can be performed using NLTK such as tokenizing, parse tree visualization, etc…


In [24]:
import gradio as gr
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import nltk
nltk.download('punkt_tab')

def summarize_text(text, min_words, max_words):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()

    # Generate an initial sentence-based summary
    summary_sentences = summarizer(parser.document, 10)  # Start with a reasonable number
    
    # Convert summary to a list of words
    summary_text = " ".join(str(sentence) for sentence in summary_sentences)
    words = summary_text.split()

    # Adjust summary length based on min & max words
    num_words = (min_words + max_words) // 2  # Use average for balanced output
    num_words = min(len(words), max(1, num_words))  # Ensure within range

    final_summary = " ".join(words[:num_words])  # Trim to word limit
    return final_summary

iface = gr.Interface(
    fn=summarize_text, 
    inputs=[
        gr.Textbox(label="Enter Text to Summarize"),
        gr.Slider(50, 20000, step=50, label="Min Summary Length (words)"),
        gr.Slider(50, 20000, step=50, label="Max Summary Length (words)")
    ], 
    outputs="text",
    title="Text Summarizer (LSA) with Word Control"
)

iface.launch(share=True)

[nltk_data] Downloading package punkt_tab to C:\Users\ASHISHI
[nltk_data]     RANJAN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


* Running on local URL:  http://127.0.0.1:7865

Could not create share link. Missing file: c:\Users\ASHISHI RANJAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\frpc_windows_amd64_v0.3. 

Please check your internet connection. This can happen if your antivirus software blocks the download of this file. You can install manually by following these steps: 

1. Download this file: https://cdn-media.huggingface.co/frpc-gradio-0.3/frpc_windows_amd64.exe
2. Rename the downloaded file to: frpc_windows_amd64_v0.3
3. Move the file to this location: c:\Users\ASHISHI RANJAN\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio
